<a href="https://colab.research.google.com/github/Tazoria/TIL/blob/main/data_analysis/basic/03_Pandas_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

# 💕 데이터 전처리

In [ ]:
df = sns.load_dataset('titanic')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


## 💡 결측치
- 결측치를 처리하는 각 방법에는 근거가 필요(근거 없이 하는 경우 위험)
- 방법
  - 삭제
  - 대치
    - 평균
    - 최빈값
    - 전후값

In [ ]:
# 결측치 확인
print(df.isnull().sum())
# deck열의 Nan 개수 계산하기
nan_deck = df['deck'].value_counts(dropna=False)
print(nan_deck)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


In [ ]:
# 결측값이 아닌 값들의 개수
df.notnull().sum(axis=1) # 행별로 확인해보기

0      14
1      15
2      14
3      15
4      14
       ..
886    14
887    15
888    13
889    15
890    14
Length: 891, dtype: int64

In [ ]:
# 결측치 확인 for반복문 으로 각 열의 Nan 개수 계산
missing_df = df.isnull()
for col in missing_df.columns:
  missing_count = missing_df[col].value_counts()
  try:
    print(col, ':', missing_count[True]) # Nan 값이 있으면 개수를 출력
  except:
    print(col, ':', missing_count[0])

survived : 891
pclass : 891
sex : 891
age : 177
sibsp : 891
parch : 891
fare : 891
embarked : 2
class : 891
who : 891
adult_male : 891
deck : 688
embark_town : 2
alive : 891
alone : 891


### 1️⃣ 삭제

In [ ]:
# Nan값이 500개 이상인 열을 모두 삭제 - deck열(891중 688개 Nan) 삭제
# threshold 옵션이 없으면 결측치를 모두 드랍, threshold 값을 넣어주면 해당 값보다 결측값이 많은 데이터들만 드랍
# 컬럼 지우기
df_tresh = df.dropna(axis=1, thresh=500)
print(len(df_tresh.columns))
print(df_tresh.columns)

14
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


In [ ]:
# 행 지우기 - 데이터를 최대한 살리기!
# 추가로 age열에 나이 데이터가 없는 모든 행을 삭제 
df_age = df_tresh.dropna(subset=['age'], how='any', axis=0)
df_age.isnull().sum() # dropna를 그냥 적용했을 때보다 4개의 값은 살림(결측치가 있는 embared(2), embark_town(2))

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       2
class          0
who            0
adult_male     0
embark_town    2
alive          0
alone          0
dtype: int64

### 2️⃣ 평균값으로 대체

In [ ]:
# age 열의 Nan값을 다른 나이 데이터의 평균으로 변경
mean_age = df['age'].mean(axis=0) # age열의 평균을 계산(NAN값을 제외)
df['age'].fillna(mean_age, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### 3️⃣ 최빈값 대체

In [ ]:
# embark_town(승선지)의 결측치를 최빈값으로 채우기
# idxmax: x축에서 최대값의 인덱스를 반환
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
print(most_freq)

# 특정 구역의 최빈값으로 대치도 가능 - 특정한경우(예: 데이터가 정렬되어있어서 인덱싱 했을 때 정확해지는경우)
# df['embark_town'][825:] 식으로 인덱싱 해주면 됨!

Southampton


In [ ]:
df['embark_town'].fillna(most_freq, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  891 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### 4️⃣ 전후방 대치

In [ ]:
# embarked 열의 Nan 값을 바로 앞에 있는 값으로 변경
# 후방대치는 'bfill'
df['embarked'].fillna(method='ffill', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  891 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


## 💡 중복 데이터
- 중복데이터 조회 : duplicated()
- 중복데이터 삭제 : drop_duplicates(subset=[])

In [ ]:
# 중복데이터를 갖는 데이터 찾기
df2 = pd.DataFrame({'c1':['a','a','b','a','b'],
                   'c2':[1,1,1,2,2],
                   'c3':[1,1,2,2,2]})
df2

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# duplicated()
df_dup = df2.duplicated()
print(df_dup)

0    False
1     True
2    False
3    False
4    False
dtype: bool


In [ ]:
# 데이터프레임의 특정 열에서 데이터 중복값을 찾기
col_dup = df2['c2'].duplicated()
print(col_dup)

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


In [ ]:
# 데이터프레임에서 중복행제거
df3 = df2.drop_duplicates()
print(df3)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [ ]:
# c2, c3열을 기준으로 중복행제거
df4 = df2.drop_duplicates(subset=['c2','c3'])
df4

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


## 💡 값 변환 (conversion)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/class/판다스data/auto-mpg.csv', header=None)
df.head()
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


### 1️⃣ 기존 값을 이용해 파생변수 생성

In [ ]:
# mpg(mile per gallon)를 kpl(kilometer per liter)로 변환하여 생성(mpg_to_kpl = 0.425)
mpg_to_kpl = 1.60934 / 3.78541

# mpg열에 0.425를 곱해서 새로운 kpl 컬럼을 생성 - 소숫점 아래 둘째자리까지 반올림
df['kpl'] = round(df['mpg'] * mpg_to_kpl, 2)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.23


### 2️⃣ 자료형 변환

In [ ]:
# 자료형 확인
df.dtypes # horsepower가 object형

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
kpl             float64
dtype: object

In [ ]:
# horse열의 고유값 확인
# 누락데이터('?') 삭제 필요
# 연속형 수치값들로 구성 - float형으로 형변환 필요
print(df['horsepower'].unique())

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']


In [ ]:
# 누락데이터('?') 삭제
df['horsepower'].replace('?', np.nan, inplace=True)
print(df.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
name            0
kpl             0
dtype: int64


In [ ]:
df.dropna(subset=['horsepower'], axis=0, inplace=True)
print(df.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
name            0
kpl             0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    object 
 4   weight        392 non-null    float64
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
 9   kpl           392 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 33.7+ KB


In [ ]:
# horsepower열 형변환(float)
df['horsepower'] = df['horsepower'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    float64
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
 9   kpl           392 non-null    float64
dtypes: float64(6), int64(3), object(1)
memory usage: 33.7+ KB


In [ ]:
# origin열의 고유값을 확인 - 범주형 데이터로 변환 필요
df['origin'].unique()

array([1, 3, 2])

In [ ]:
df['origin'].replace({1:'USA', 2:'EU', 3:'JAPAN'}, inplace=True)
df['origin'].unique()

array(['USA', 'JAPAN', 'EU'], dtype=object)

In [ ]:
# 타입 확인해보기
print(df['origin'].dtypes)

object


In [ ]:
# origin 열의 문자열 자료형을 범주형으로 변환 - astype('str')을 해주면 카테고리형을 반대로 오브젝트형으로 변환 가능
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtypes)

category


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    int64   
 7   origin        392 non-null    category
 8   name          392 non-null    object  
 9   kpl           392 non-null    float64 
dtypes: category(1), float64(6), int64(2), object(1)
memory usage: 31.1+ KB


In [ ]:
# model year에서 랜덤으로 3개를 뽑아 생김새 확인하기
df['model year'].sample(3) # sample()적용시 [인덱스, 값]으로 출력

277    78
87     73
67     72
Name: model year, dtype: int64

In [ ]:
# model year정수를 범주형으로 변환(model year: 76, 80 등)
df['model year'] = df['model year'].astype('category')
df['model year'].sample(3)

388    82
81     72
130    74
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]

### 3️⃣ 수치형데이터를 범주화(feat. cut( ), np.histogram( , bins= ))

#### 🔘 단순 범주화 : np.histogram( , bins= ) & cut( )

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    category
 7   origin        392 non-null    category
 8   name          392 non-null    object  
 9   kpl           392 non-null    float64 
dtypes: category(2), float64(6), int64(1), object(1)
memory usage: 29.1+ KB


In [ ]:
# horsepower를 3개의 등급으로 나누기
# np.histogram 함수로 3개의 bin으로 나누는 경계 값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [ ]:
# 3개의 bins에 이름 지정
bin_name = ['저출력', '보통출력', '고출력']
# cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],
                      bins=bin_dividers, # 경계값 리스트
                      labels=bin_name, # 해당구간 이름
                      include_lowest=True) # 첫 경계값의 포함여부)
print(df[['horsepower', 'hp_bin']].head(10))

   horsepower hp_bin
0       130.0   보통출력
1       165.0   보통출력
2       150.0   보통출력
3       150.0   보통출력
4       140.0   보통출력
5       198.0    고출력
6       220.0    고출력
7       215.0    고출력
8       225.0    고출력
9       190.0    고출력


#### 🔘 원핫인코딩

In [ ]:
df_dummies = pd.get_dummies(df, columns=['hp_bin'])
df_dummies

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl,hp_bin_저출력,hp_bin_보통출력,hp_bin_고출력
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65,0,1,0
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38,0,1,0
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65,0,1,0
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80,0,1,0
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,USA,ford mustang gl,11.48,1,0,0
394,44.0,4,97.0,52.0,2130.0,24.6,82,EU,vw pickup,18.71,1,0,0
395,32.0,4,135.0,84.0,2295.0,11.6,82,USA,dodge rampage,13.60,1,0,0
396,28.0,4,120.0,79.0,2625.0,18.6,82,USA,ford ranger,11.90,1,0,0


In [ ]:
from sklearn import preprocessing
# LabelEnocder
label_encoder = preprocessing.LabelEncoder()

onehot_labeled = df.copy()
onehot_labeled['hp_bin'] = label_encoder.fit_transform(onehot_labeled['hp_bin'])
onehot_labeled.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl,hp_bin
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23,1


In [ ]:
# OneHotEncoder
# onehot_encoder = preprocessing.OneHotEncoder()

# onehot = df.copy()
# onehot['hp_bin'] = label_encoder.fit_transform(onehot['hp_bin'])
# onehot = onehot_encoder.fit_transform(onehot)
# onehot

<392x1194 sparse matrix of type '<class 'numpy.float64'>'
	with 4312 stored elements in Compressed Sparse Row format>

## 💡 시계열

In [ ]:
# 주식데이터(2018.06.01~2018.07.02)
df = pd.read_csv('/content/drive/MyDrive/class/판다스data/stock-data.csv')
df.head()

,Date,Close,Start,High,Low,Volume
0,2018-07-02,10100,10850,10900,10000,137977
1,2018-06-29,10700,10550,10900,9990,170253
2,2018-06-28,10400,10900,10950,10150,155769
3,2018-06-27,10900,10800,11050,10500,133548
4,2018-06-26,10800,10900,11000,10700,63039


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


In [ ]:
# 날짜를 오브젝트 -> 날짜 타입으로 변경
# 날짜를 인덱스로 변경
df['new_date'] = pd.to_datetime(df['Date'])
df.set_index('new_date', inplace=True)
df.drop('Date', axis=1, inplace=True)
df.head()

,Close,Start,High,Low,Volume
new_date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


In [ ]:
dates = ['2019-01-01', '2020-03-01', '2021-06-01']
ts_dates = pd.to_datetime(dates)
print(ts_dates)

# Timestamp를 period로 변환
pr_day = ts_dates.to_period(freq='D')
print(pr_day)
pr_month = ts_dates.to_period(freq='M')
print(pr_day)
pr_year = ts_dates.to_period(freq='A')
print(pr_day)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)
PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')


In [ ]:
# timestamp의 배열 만들기(날짜 범위 만들기) - 월간격, 월의 시작일 기준
ts_ms = pd.date_range(start='2019-01-01', # start: 시작날짜
                      end=None, # end:종료날짜
                      periods=6,
                      freq='MS', # freq: 시간 간격 ('MS': 월의 시작일)
                      tz='Asia/Seoul' # 시간대 지역
                      )

print(len(ts_ms))
print(ts_ms)

6
DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [ ]:
# 월간격, 월의 마지막 날 기준
ts_me = pd.date_range(start='2019-01-01',
                      periods=6,
                      freq='M',
                      tz='Asia/Seoul',)
print(len(ts_me))
print(ts_me)

6
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


In [ ]:
# 분기(3개월), 간격, 월의 마지막날 기준
ts_3m = pd.date_range(start='2019-01-01',
                      periods=6,
                      freq='3M',
                      tz='Asia/Seoul',)
print(len(ts_3m))
print(ts_3m)

6
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')
